In [1]:
import torch 

from la.utils.io_utils import load_data 
from la.utils.io_utils import preprocess_dataset

/home/don/miniconda3/envs/lf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/don/miniconda3/envs/lf/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/don/miniconda3/envs/lf/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


2023-10-01 16:11:37 INFO     Created a temporary directory at              ]8;id=123570;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/torch/distributed/nn/jit/instantiator.py\torch.distributed.nn.jit.instantiator]8;;\:]8;id=942951;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/torch/distributed/nn/jit/instantiator.py#21\21]8;;\
                             /tmp/tmpwanpx5hy                                                                      

                    INFO     Writing                                       ]8;id=321062;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/torch/distributed/nn/jit/instantiator.py\torch.distributed.nn.jit.instantiator]8;;\:]8;id=790545;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/torch/distributed/nn/jit/instantiator.py#76\76]8;;\
                             /tmp/tmpwanpx5hy/_remote_module_non_scriptabl                                         
                             e.py                                                                                  

In [2]:
%load_ext autoreload
%autoreload 2

import hydra
from hydra import initialize, compose
from typing import Dict, List
from la.data.my_dataset_dict import MyDatasetDict

initialize(version_base=None, config_path=str("../conf"), job_name="relative_supervision")

hydra.initialize()

In [3]:
import logging


pylogger = logging.getLogger(__name__)

In [4]:
from nn_core.common import PROJECT_ROOT

# Instantiate torchvision dataset
cfg = compose(config_name="relative_supervision", overrides=[])

In [5]:
dataset = load_data(cfg)

dataset = preprocess_dataset(dataset, cfg)

/home/don/miniconda3/envs/lf/lib/python3.9/site-packages/datasets/load.py:2072: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(
Converting to RGB: 100%|██████████| 10000/10000 [00:05<00:00, 1909.50 examples/s]


In [6]:
img_size = dataset['train'][0]['x'].shape[0]
pylogger.info(f"Image size: {img_size}")
pylogger.info(f"Train shape: {dataset['train'][0]['x'].shape}")

2023-10-01 16:12:15 INFO     Image size: 32                                                              ]8;id=105141;file:///tmp/ipykernel_100307/276542090.py\__main__]8;;\:]8;id=787834;file:///tmp/ipykernel_100307/276542090.py#2\2]8;;\

                    INFO     Train shape: (32, 32, 3)                                                    ]8;id=762221;file:///tmp/ipykernel_100307/276542090.py\__main__]8;;\:]8;id=797593;file:///tmp/ipykernel_100307/276542090.py#3\3]8;;\

In [7]:
import pytorch_lightning as pl

teacher: pl.LightningModule = hydra.utils.instantiate(
    cfg.teacher,
    _recursive_=False,
    num_classes=cfg.dataset.num_classes,
    model=cfg.teacher.model,
    input_dim=img_size,
)

                    INFO     Loading pretrained weights from Hugging Face hub              ]8;id=892930;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/timm/models/_builder.py\timm.models._builder]8;;\:]8;id=653879;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/timm/models/_builder.py#182\182]8;;\
                             (timm/efficientnet_b0.ra_in1k)                                                        

2023-10-01 16:12:16 INFO     [timm/efficientnet_b0.ra_in1k] Safe alternative available for     ]8;id=397124;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/timm/models/_hub.py\timm.models._hub]8;;\:]8;id=714581;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/timm/models/_hub.py#180\180]8;;\
                             'pytorch_model.bin' (as 'model.safetensors'). Loading weights                         
                             using safetensors.                                                                    

In [8]:
import numpy as np 
subsample = False
if subsample:
    modes = ['train', 'test']
    for mode in modes:
        dataset[mode] = dataset[mode].select(np.arange(100))

In [9]:
map_params = {
            "function": lambda x: {"x": teacher.transform_func(x["x"])},
            "writer_batch_size": 100,
            "num_proc": 1,
        }

for mode in modes:
    dataset[mode] = dataset[mode].map(
        desc=f"Transforming {mode} samples", **map_params
    )

Transforming test samples: 100%|██████████| 100/100 [00:11<00:00,  8.44 examples/s]


In [10]:
dataset.set_format(type="torch", columns=['x', 'y'])

# Teacher

In [11]:
from functools import partial 
from pytorch_lightning import Trainer

loader_func = partial(
            torch.utils.data.DataLoader,
            batch_size=512,
            num_workers=8,
)

# TODO: switch back num epochs
trainer_func = partial(Trainer, gpus=1, max_epochs=30, logger=False, enable_progress_bar=True)

In [12]:
train_loader = loader_func(dataset['train'], shuffle=True)
test_loader = loader_func(dataset['test'], shuffle=False)

In [13]:
trainer = trainer_func()
trainer.fit(teacher, train_loader)

/home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(


2023-10-01 16:12:42 INFO     GPU available: True (cuda), used: True      ]8;id=461532;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=388977;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1751\1751]8;;\

                    INFO     TPU available: False, using: 0 TPU cores    ]8;id=104524;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=981358;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1754\1754]8;;\

                    INFO     IPU available: False, using: 0 IPUs         ]8;id=758785;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=698661;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1757\1757]8;;\

                    INFO     HPU available: False, using: 0 HPUs         ]8;id=679348;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=807410;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1760\1760]8;;\

...

2023-10-01 16:12:43 INFO     LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]       ]8;id=791600;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py\pytorch_lightning.accelerators.cuda]8;;\:]8;id=23539;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py#57\57]8;;\

                    INFO                                               ]8;id=911858;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_summary.py\pytorch_lightning.callbacks.model_summary]8;;\:]8;id=44145;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_summary.py#83\83]8;;\
                               | Name           | Type                                                             
                             | Params                                                                              
                             -----------------------------------------                                             
                             --------------                                                                        
                             0 | train_accuracy | Accuracy                                                         
                             | 0                                                                                   
                             1 | val_accuracy   | Accuracy                                                         
                             | 0                                                                                   
                             2 | test_accuracy  | Accuracy                                                         
                             | 0                                                                                   
                             3 | data_augm      | DataAugmentation                                                 
                             | 0                                                                                   
                             4 | embedder       | EfficientNetWrapper                                              
                             | 4.0 M                                                                               
                             5 | projector      | VanillaMLPProjector                                              
                             | 360 K                                                                               
                             6 | classifier     | Linear                                                           
                             | 12.9 K                                                                              
                             -----------------------------------------                                             
                             --------------                                                                        
                             373 K     Trainable params                                                            
                             4.0 M     Non-trainable params                                                        
                             4.4 M     Total params                                                                
                             17.525    Total estimated model params                                                
                             size (MB)                                                                             

Epoch 29: 100%|██████████| 1/1 [00:01<00:00,  1.82s/it, loss=2.03, loss/train=0.953]

2023-10-01 16:13:42 INFO     `Trainer.fit` stopped: `max_epochs=30`       ]8;id=769972;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=676401;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py#170\170]8;;\
                             reached.                                                                              

Epoch 29: 100%|██████████| 1/1 [00:01<00:00,  1.89s/it, loss=2.03, loss/train=0.953]


In [14]:
trainer.test(teacher, test_loader)

                    INFO     LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]       ]8;id=207002;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py\pytorch_lightning.accelerators.cuda]8;;\:]8;id=270512;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py#57\57]8;;\

Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  3.18it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         acc/test          │    0.20999999344348907    │
│         loss/test         │     5.043737888336182     │
└───────────────────────────┴───────────────────────────┘

[{'loss/test': 5.043737888336182, 'acc/test': 0.20999999344348907}]

# Student

In [15]:
img_size = dataset['train'][0]['x'].shape[-1]
img_size

224

In [16]:
student: pl.LightningModule = hydra.utils.instantiate(
    cfg.student,
    _recursive_=False,
    num_classes=cfg.dataset.num_classes,
    model=cfg.student.model,
    input_dim=img_size,
)

2023-10-01 16:13:44 INFO     shapes: torch.Size([1, 16, 7, 7])                    ]8;id=970925;file:///home/don/Code/latent-aggregation/src/la/modules/vanilla_cnn_student.py\la.modules.vanilla_cnn_student]8;;\:]8;id=909872;file:///home/don/Code/latent-aggregation/src/la/modules/vanilla_cnn_student.py#55\55]8;;\

In [17]:
train_loader = loader_func(dataset['train'], shuffle=True)
test_loader = loader_func(dataset['test'], shuffle=False)

In [18]:
trainer = trainer_func()
trainer.fit(student, train_loader)

2023-10-01 16:13:45 INFO     GPU available: True (cuda), used: True      ]8;id=559891;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=536913;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1751\1751]8;;\

                    INFO     TPU available: False, using: 0 TPU cores    ]8;id=33090;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=774181;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1754\1754]8;;\

                    INFO     IPU available: False, using: 0 IPUs         ]8;id=161641;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=636526;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1757\1757]8;;\

                    INFO     HPU available: False, using: 0 HPUs         ]8;id=819803;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=459936;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1760\1760]8;;\

                    INFO     LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]       ]8;id=956906;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py\pytorch_lightning.accelerators.cuda]8;;\:]8;id=706827;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py#57\57]8;;\

                    INFO                                               ]8;id=594412;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_summary.py\pytorch_lightning.callbacks.model_summary]8;;\:]8;id=503210;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_summary.py#83\83]8;;\
                               | Name           | Type             |                                               
                             Params                                                                                
                             -----------------------------------------                                             
                             -----------                                                                           
                             0 | train_accuracy | Accuracy         | 0                                             
                             1 | val_accuracy   | Accuracy         | 0                                             
                             2 | test_accuracy  | Accuracy         | 0                                             
                             3 | data_augm      | DataAugmentation | 0                                             
                             4 | model          | StudentCNN       |                                               
                             127 K                                                                                 
                             -----------------------------------------                                             
                             -----------                                                                           
                             127 K     Trainable params                                                            
                             0         Non-trainable params                                                        
                             127 K     Total params                                                                
                             0.510     Total estimated model params                                                
                             size (MB)                                                                             

Epoch 29: 100%|██████████| 1/1 [00:02<00:00,  2.04s/it, loss=3.78, loss/train=3.420]

2023-10-01 16:14:37 INFO     `Trainer.fit` stopped: `max_epochs=30`       ]8;id=60856;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=419118;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py#170\170]8;;\
                             reached.                                                                              

Epoch 29: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it, loss=3.78, loss/train=3.420]


In [19]:
trainer.test(student, test_loader)

                    INFO     LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]       ]8;id=268241;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py\pytorch_lightning.accelerators.cuda]8;;\:]8;id=189134;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py#57\57]8;;\

Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 40.05it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         acc/test          │   0.029999999329447746    │
│         loss/test         │      4.9328932762146      │
└───────────────────────────┴───────────────────────────┘

[{'loss/test': 4.9328932762146, 'acc/test': 0.029999999329447746}]

# Standard distillation

In [20]:
from tqdm import tqdm


def embed_samples(dataset, model) -> Dict:

    model = model.to("cuda")
    dataloader = loader_func(dataset, shuffle=False, batch_size=8)

    all_embeddings = []
    all_logits = []

    for batch in tqdm(dataloader, desc=f"Embedding {mode} samples"):
        x = batch["x"].to("cuda")
        model_out = model(x)
        embeds, logits = model_out["embeds"].detach().cpu(), model_out["logits"].detach().cpu()
        all_embeddings.extend(embeds)
        all_logits.extend(logits)

    all_embeddings = torch.stack(all_embeddings)
    all_logits = torch.stack(all_logits)

    map_params = {
        "with_indices": True,
        "batched": True,
        "batch_size": 128,
        "num_proc": 1,
        "writer_batch_size": 10,
    }

    dataset = dataset.map(
        function=lambda x, ind: {
            "embedding": all_embeddings[ind],
        },
        desc=f"Storing embedded samples",
        **map_params,
    )
    dataset = dataset.map(
        function=lambda x, ind: {
            "logits": all_logits[ind],
        },
        desc=f"Storing logits",
        **map_params,
    )

    return dataset

In [21]:
dataset['train'] = embed_samples(dataset['train'], teacher)
dataset['test'] = embed_samples(dataset['test'], teacher)

Storing logits: 100%|██████████| 100/100 [00:01<00:00, 79.64 examples/s]


In [22]:
dataset['train'].set_format(type="torch", columns=['x', 'y', 'embedding', 'logits'])
dataset['test'].set_format(type="torch", columns=['x', 'y', 'embedding', 'logits'])

# Stan

In [23]:
num_anchors = 99
anchor_idxs = np.random.choice(len(dataset['train']), size=num_anchors, replace=False)

anchors = dataset['train'].select(anchor_idxs)

In [24]:
relative_student = hydra.utils.instantiate(
    cfg.student_relative,
    _recursive_=False,
    num_classes=cfg.dataset.num_classes,
    model=cfg.student_relative.model,
    input_dim=img_size,
    anchors=anchors
)

2023-10-01 16:14:46 INFO     shapes: torch.Size([1, 16, 7, 7])                    ]8;id=903327;file:///home/don/Code/latent-aggregation/src/la/modules/vanilla_cnn_student.py\la.modules.vanilla_cnn_student]8;;\:]8;id=283981;file:///home/don/Code/latent-aggregation/src/la/modules/vanilla_cnn_student.py#55\55]8;;\

In [25]:
train_loader = loader_func(dataset['train'], shuffle=True)
test_loader = loader_func(dataset['test'], shuffle=False)

In [26]:
trainer = trainer_func()
trainer.fit(relative_student, train_loader)

2023-10-01 16:14:48 INFO     GPU available: True (cuda), used: True      ]8;id=794443;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=984853;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1751\1751]8;;\

                    INFO     TPU available: False, using: 0 TPU cores    ]8;id=643975;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=78739;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1754\1754]8;;\

                    INFO     IPU available: False, using: 0 IPUs         ]8;id=657426;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=646280;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1757\1757]8;;\

                    INFO     HPU available: False, using: 0 HPUs         ]8;id=704980;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=780428;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1760\1760]8;;\

                    INFO     LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]       ]8;id=472206;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py\pytorch_lightning.accelerators.cuda]8;;\:]8;id=820932;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py#57\57]8;;\

                    INFO                                               ]8;id=800691;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_summary.py\pytorch_lightning.callbacks.model_summary]8;;\:]8;id=313821;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_summary.py#83\83]8;;\
                               | Name              | Type                                                          
                             | Params                                                                              
                             -----------------------------------------                                             
                             --------------                                                                        
                             0 | train_accuracy    | Accuracy                                                      
                             | 0                                                                                   
                             1 | val_accuracy      | Accuracy                                                      
                             | 0                                                                                   
                             2 | test_accuracy     | Accuracy                                                      
                             | 0                                                                                   
                             3 | data_augm         | DataAugmentation                                              
                             | 0                                                                                   
                             4 | model             | StudentCNN                                                    
                             | 127 K                                                                               
                             5 | distillation_loss | MSELoss                                                       
                             | 0                                                                                   
                             -----------------------------------------                                             
                             --------------                                                                        
                             127 K     Trainable params                                                            
                             0         Non-trainable params                                                        
                             127 K     Total params                                                                
                             0.510     Total estimated model params                                                
                             size (MB)                                                                             

Epoch 29: 100%|██████████| 1/1 [00:02<00:00,  2.03s/it, loss=4.76, loss/train=4.690]

2023-10-01 16:15:45 INFO     `Trainer.fit` stopped: `max_epochs=30`       ]8;id=481964;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=860210;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py#170\170]8;;\
                             reached.                                                                              

Epoch 29: 100%|██████████| 1/1 [00:02<00:00,  2.20s/it, loss=4.76, loss/train=4.690]


In [27]:
trainer.test(relative_student, test_loader)

                    INFO     LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]       ]8;id=523983;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py\pytorch_lightning.accelerators.cuda]8;;\:]8;id=775430;file:///home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py#57\57]8;;\

Testing: 0it [00:00, ?it/s]

/home/don/miniconda3/envs/lf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
